# Gather Package Links

This notebook is used to collect the links to the Linux kernel packages for different Ubuntu versions.

Note: You can safely **skip this notebook**, and refer to the output of the notebook to get the links.

In [4]:
from launchpadlib.launchpad import Launchpad  # Ref: https://api.launchpad.net/1.0/

lp = Launchpad.login_anonymously("ubuntu-build-list", "production")
ubuntu = lp.distributions["ubuntu"]  # type: ignore
archive = ubuntu.main_archive

versions = {
    s.version: s
    for s in ubuntu.series
    if (16, 4) <= tuple(int(part) for part in s.version.split(".")) <= (24, 4)
}
versions

{'24.04': <distro_series at https://api.launchpad.net/1.0/ubuntu/noble>,
 '23.10': <distro_series at https://api.launchpad.net/1.0/ubuntu/mantic>,
 '23.04': <distro_series at https://api.launchpad.net/1.0/ubuntu/lunar>,
 '22.10': <distro_series at https://api.launchpad.net/1.0/ubuntu/kinetic>,
 '22.04': <distro_series at https://api.launchpad.net/1.0/ubuntu/jammy>,
 '21.10': <distro_series at https://api.launchpad.net/1.0/ubuntu/impish>,
 '21.04': <distro_series at https://api.launchpad.net/1.0/ubuntu/hirsute>,
 '20.10': <distro_series at https://api.launchpad.net/1.0/ubuntu/groovy>,
 '20.04': <distro_series at https://api.launchpad.net/1.0/ubuntu/focal>,
 '19.10': <distro_series at https://api.launchpad.net/1.0/ubuntu/eoan>,
 '19.04': <distro_series at https://api.launchpad.net/1.0/ubuntu/disco>,
 '18.10': <distro_series at https://api.launchpad.net/1.0/ubuntu/cosmic>,
 '18.04': <distro_series at https://api.launchpad.net/1.0/ubuntu/bionic>,
 '17.10': <distro_series at https://api.lau

In [5]:
import re
from pathlib import Path
from typing import Optional, Tuple, List


def get_ddeb(
    distro_series,
    source_name: str = "linux",
    pocket: str = "Release",
    arch: Optional[str] = None,
    flavor: Optional[str] = None,
) -> Tuple[str, List[str]]:
    published_sources = archive.getPublishedSources(
        exact_match=True,
        source_name=source_name,
        distro_series=distro_series,
        pocket=pocket,
        order_by_date=True,
    )

    if len(published_sources) == 0:
        return "", []

    versions = [
        tuple(int(part) for part in re.split(r"-|\.|\+", s.source_package_version))
        for s in published_sources
    ]
    if max(versions) != versions[0]:
        raise ValueError(f"The latest version is not the first: {versions}")

    source = published_sources[0]
    web_link = f"https://launchpad.net/ubuntu/+source/{source_name}/{source.source_package_version}"

    urls = []
    for url in source.binaryFileUrls():
        path = Path(url)
        if path.suffix != ".ddeb":
            continue
        if not path.stem.startswith("linux-image"):
            continue
        if arch is not None and not path.stem.endswith(arch):
            continue
        if flavor is not None and flavor not in path.stem:
            continue
        urls.append(url)
    return web_link, urls

In [6]:
# https://launchpad.net/ubuntu/+source/linux/<version>

for v in versions.values():
    web_link, urls = get_ddeb(v, arch="amd64", flavor="generic")
    assert len(urls) == 1, urls
    print(v.version, urls[0])

24.04 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-31-generic-dbgsym_6.8.0-31.31_amd64.ddeb
23.10 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.5.0-9-generic-dbgsym_6.5.0-9.9_amd64.ddeb
23.04 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.2.0-20-generic-dbgsym_6.2.0-20.20_amd64.ddeb
22.10 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.19.0-21-generic-dbgsym_5.19.0-21.21_amd64.ddeb
22.04 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.15.0-25-generic-dbgsym_5.15.0-25.25_amd64.ddeb
21.10 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.13.0-19-generic-dbgsym_5.13.0-19.19_amd64.ddeb
21.04 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.11.0-16-generic-dbgsym_5.11.0-16.17_amd64.ddeb
20.10 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.8.0-25-generic-dbgsym_5.

In [12]:
# https://launchpad.net/ubuntu/+source/<source_name>
# https://ubuntu.com/kernel/variants
for v in ["20.04", "24.04"]:
    print(f"{v:-^80}")
    for source in [
        # The default linux
        "linux",
        # Cloud-optimized kernels: https://ubuntu.com/blog/cloud-optimized-linux-kernels
        "linux-aws",
        "linux-azure",
        "linux-gcp",
        "linux-oracle",
        "linux-gke",
        "linux-ibm",
        # Other flavors
        "linux-riscv",
        "linux-raspi",
        "linux-lowlatency",  # Used to be part of `linux`
        "linux-kvm",  # No debug symbols, discontinued after 22.04
        "linux-laptop",  # New in 24.04, arm64 only
        "linux-nvidia",  # New in 22.04
    ]:
        web_link, urls = get_ddeb(versions[v], source_name=source)
        print(f"{source:20}{web_link}")
        for url in urls:
            print(f"{'':20}{url}")

-------------------------------------20.04--------------------------------------
linux               https://launchpad.net/ubuntu/+source/linux/5.4.0-26.30
                    https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_s390x.ddeb
                    https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_ppc64el.ddeb
                    https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_amd64.ddeb
                    https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-lowlatency-dbgsym_5.4.0-26.30_amd64.ddeb
                    https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_arm64.ddeb
                    https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-5.4.0-26-generic-dbgsym_5.4.0-26.30_armhf.ddeb
  